In [7]:
# !pip install pydub
# !pip install azure-cognitiveservices-speech

In [1]:
import azure.cognitiveservices.speech as speechsdk
from pydub import AudioSegment
import os

subscription_key = os.getenv('SUBSCRIPTION_KEY')
service_region = os.getenv('SERVICE_REGION')
translator_endpoint = "https://api.cognitive.microsofttranslator.com"
speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=service_region)

c:\Users\kaharlyr\AppData\Local\miniconda3\envs\webapp\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
# Specify the file path
# text_sample_file_path = "L_and_D_text_sample.txt"
text_sample_file_path = "comms_text_sample_en.txt"


# Read the text from the file and store it in the text_prompt variable
with open(text_sample_file_path, "r", encoding="utf-8") as file:
    text_prompt = file.read()


In [5]:
text_prompt

"Hello and thank you for calling the WSIB. Our office is closed, and we’re sorry we missed your call. If you're in crisis and need immediate support, call Telehealth Ontario at 1-866-797-0000. In an emergency always call 911. If you know the 10-digit phone number of the person you wish to reach and would like to leave a message, press 1. You can reach us weekdays between 7:30 a.m. and 6:00 p.m. You can also visit wsib.ca/onlineservices to find all the ways to work with us online, 24/7."

In [7]:
# voice_styles = {
#     "Cora": ["Default"],
#     "Nancy": ["Friendly"]
# }

In [12]:
voice_styles = {
    # "Sylvie": ["Default"],
    "Brigitte": ["Default"],
    "Celeste": ["Friendly"]
}

In [13]:
# voice_styles = {
#     "Aria": ["Customer service", "Newscast-Casual", "Narration-professional", "Friendly", "Chat"],
#     "Jason": ["Default", "Cheerful"],
#     "Jenny": ["Default", "Customer service", "Newscast", "Assistant", "Friendly"],
#     "Guy": ["Newscast", "Friendly"]
# }

In [16]:
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=None)

low_quality_also = True # set True if you want 8kHz version too

for voice_name in voice_styles:
    for style in voice_styles[voice_name]:
        style = style.lower()

        ssml_string = """
        <!--ID=B7267351-473F-409D-9765-754A8EBCDE05;Version=1|{"VoiceNameToIdMapItems":[{"Id":"27e2f1c8-cfe0-4324-88e2-cd0bafeffe1b","Name":"Microsoft Server Speech Text to Speech Voice (en-US, %sNeural)","ShortName":"en-US-%sNeural","Locale":"en-US","VoiceType":"StandardVoice"}]}-->
        <!--ID=5B95B1CC-2C7B-494F-B746-CF22A0E779B7;Version=1|{"Locales":{"en-US":{"AutoApplyCustomLexiconFiles":[{}]},"de-DE":{"AutoApplyCustomLexiconFiles":[{}]},"fr-FR":{"AutoApplyCustomLexiconFiles":[{}]},"en-CA":{"AutoApplyCustomLexiconFiles":[{}]}}}-->
        <!--ID=FCB40C2B-1F9F-4C26-B1A1-CF8E67BE07D1;Version=1|{"Files":{}}-->
        <speak xmlns="http://www.w3.org/2001/10/synthesis" xmlns:mstts="http://www.w3.org/2001/mstts" xmlns:emo="http://www.w3.org/2009/10/emotionml" version="1.0" xml:lang="en-US"><voice name="en-US-%sNeural"><s /><mstts:express-as style="%s">%s</mstts:express-as><s /></voice></speak>
        """ % (voice_name, voice_name, voice_name, style, text_prompt)
        ssml_string = ssml_string.strip()

        
        result = speech_synthesizer.speak_ssml_async(ssml_string).get()

        stream = speechsdk.AudioDataStream(result)

        file_name_24kHz = f"generated_samples/24kHz/{voice_name}_{style}_24kHz.wav"
        stream.save_to_wav_file(file_name_24kHz)

        print("Processed", file_name_24kHz)

        if low_quality_also:
            # Load the generated audio file (adjust the filename as needed)
            audio = AudioSegment.from_wav(file_name_24kHz)

            # Set the desired sample rate to 8 kHz
            desired_sample_rate = 8000

            # Resample the audio to the desired sample rate
            audio = audio.set_frame_rate(desired_sample_rate)
            file_name_8kHz = f"generated_samples/8kHz/{voice_name}_{style}_8kHz.wav"
            # Export the resampled audio to a new file
            audio.export(file_name_8kHz, format="wav")




ValueError: Invalid format specifier '"27e2f1c8-cfe0-4324-88e2-cd0bafeffe1b","Name":"Microsoft Server Speech Text to Speech Voice (fr-Fr, BrigitteNeural)","ShortName":"fr-Fr-BrigitteNeural","Locale":"fr-Fr","VoiceType":"StandardVoice"' for object of type 'str'